# Predict

In [1]:
# Parameters cells
WITNESS_NAME = "CHSH"
SIMULATION_PATH = "./simulated_data"
MLFLOW_URL = "http://localhost:5000"
AIRFLOW_DAG_RUN_ID = "test-dm-chsh"
MLFLOW_RUN_ID = "1c81e50024c54dd69d8e48bc406c4dad"
AWS_ACCESS_KEY_ID="minio123"
AWS_SECRET_ACCESS_KEY="minio123"
MLFLOW_S3_ENDPOINT_URL="http://localhost:9990"

In [2]:
from os import environ

environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "false"

In [3]:
import mlflow

if MLFLOW_URL is not None:
    environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
    mlflow.set_tracking_uri(MLFLOW_URL)

In [4]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf

if MLFLOW_RUN_ID is None:
    client = mlflow.tracking.MlflowClient()

    experiments = client.search_experiments(
        filter_string = "name = 'ML Quantum Entanglement'"
    )
    experiment_id = experiments[0].experiment_id

    runs = client.search_runs(
        experiment_ids=[experiment_id], 
        filter_string=f"tags.airflow_dag_run_id = '{AIRFLOW_DAG_RUN_ID}'"
    )
    run_id = runs[0].info.run_id
else :
    run_id = MLFLOW_RUN_ID

# Replace '<run_id>' with the actual run ID where your model is stored
model_uri = f"runs:/{run_id}/model"

# Load the TensorFlow model from MLflow
model = mlflow.tensorflow.load_model(model_uri)

2024-06-02 23:51:53.059224: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-02 23:51:53.093318: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 23:51:53.671581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-02 23:51:54.291583: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-06-02 23:51:54.291603: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving 

In [18]:
from simulation_utils import flatten_density_matrix, create_random_separable, create_bell_states


for bell_dm in create_bell_states():
    flatten_bell = flatten_density_matrix(bell_dm)
    predictions = model.predict(tf.constant([flatten_bell]))
    print(f"Bell state \n{bell_dm}: \nentanglement prediction {predictions}")
    assert predictions[0][0] > 0.9999

for _ in range(0, 4):
    separable_dm = create_random_separable()
    flatten_separable = flatten_density_matrix(separable_dm)
    predictions = model.predict(tf.constant([flatten_separable]))
    print(f"Separable state {flatten_separable}: \nentanglement prediction {predictions}")
    assert predictions[0][0] > 0 and predictions[0][0] < 1e-5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Bell state 
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]: 
entanglement prediction [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Bell state 
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.5  0.   0.  -0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [-0.5  0.   0.   0.5]]: 
entanglement prediction [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Bell state 
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.  0.  0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  0.  0. ]]: 
entanglement prediction [[0.99998856]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Bell state 
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.   0.   0.   0. ]
 [ 0.   0.5